In [49]:
# https://www.youtube.com/watch?v=i3uMhH2xeOM&ab_channel=Buynomics

In [50]:
# https://ngugijoan.medium.com/pricing-on-point-the-art-and-science-of-dynamic-pricing-dd543bf80f01
# https://ngugijoan.medium.com/dynamic-pricing-implementation-through-data-science-price-optimization-strategies-56adab4d3176
# https://levelup.gitconnected.com/calculating-individual-price-elasticity-for-products-9787e3b82875
# https://www.kaggle.com/code/arnabchaki/flight-fare-prediction-0-96-r2-score?fbclid=IwZXh0bgNhZW0CMTAAAR05L4by3xyhImYsDOnF-ufsQQ7VbBefv8Bg3ECHy1JHCR_XmjSZIAKM7yE_aem_AWdIsN4qMSlU9R0FQsAR9y8hT_e_ggs_tIfnGdUdpwA4mLwPAbLPidigOsMMcKNF-4wyLjSg2hcmqzefdb3gX5bT
# https://datascience.oneoffcoder.com/pricing-elasticity-modeling.html#Random-forest

In [51]:
import warnings
warnings.simplefilter("ignore")

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

In [53]:
import duckdb

In [54]:
from sklearn.tree import DecisionTreeRegressor

In [55]:
from sklearn.model_selection import GridSearchCV

In [56]:
from sklearn.metrics import make_scorer, mean_squared_error

# 10. Load data

In [57]:
df = pd.read_csv("online_retail_II_cleaned_with_cost_and_cat.csv")

In [58]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,profit_margin,Cost_price,Category
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,0.193525,5.605004,Home and Garden
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,0.475850,3.538012,Home and Garden
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,0.368677,4.261430,Home and Garden
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,0.303343,1.462980,Arts and Crafts
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,0.086449,1.141939,Arts and Crafts


In [59]:
df.shape

(397432, 11)

# 11. Feature engineering

In [60]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [61]:
df['yyyymmdd'] = df['InvoiceDate'].dt.strftime('%Y%m%d')

datetime features

In [62]:
def datetime_feature_extraction(df):
    # Day extraction
    df['dayofweek'] = df['InvoiceDate'].dt.dayofweek
    df['dayofmonth'] = df['InvoiceDate'].dt.day
    df['dayofyear'] = df['InvoiceDate'].dt.dayofyear
    df['is_weekend'] = (df['InvoiceDate'].dt.dayofweek >= 5).astype(int)
    df['weekday_weekend'] = df['InvoiceDate'].dt.dayofweek.apply(lambda x: 0 if x >= 5 else 1)

    # Week extraction
    df['weekofyear'] = df['InvoiceDate'].dt.isocalendar().week
    df['weekofmonth'] = np.ceil(df['dayofmonth'] / 7).astype(int)    
    
    # Month extraction
    df['month'] = df['InvoiceDate'].dt.month
    df['quarter'] = df['InvoiceDate'].dt.quarter
    df['days_in_month'] = df['InvoiceDate'].dt.days_in_month

    # Year extraction
    df['year'] = 2019 - df['InvoiceDate'].dt.year
    return df

In [63]:
df = datetime_feature_extraction(df)

group by

In [64]:
aggregation_functions = {
    'Quantity': 'sum',
    'Price': 'mean',
    'Cost_price':'mean',
    'dayofweek': 'max',
    'dayofmonth': 'max',
    'dayofyear': 'max',
    'is_weekend': 'max',
    'weekday_weekend': 'max',
    'weekofyear': 'max',
    'weekofmonth': 'max',
    'month': 'max',
    'quarter': 'max',
    'days_in_month': 'max',
    'year': 'max'
}

In [65]:
df = df.groupby(['StockCode','Category','Description','yyyymmdd']).agg(aggregation_functions).reset_index()

create change

In [66]:
df['change_qty'] = df['Quantity'].pct_change()

In [67]:
df['change_price'] = df['Price'].pct_change()

In [68]:
df.dropna(inplace=True)

In [69]:
df.sample(10)

,StockCode,Category,Description,yyyymmdd,Quantity,Price,Cost_price,dayofweek,dayofmonth,dayofyear,is_weekend,weekday_weekend,weekofyear,weekofmonth,month,quarter,days_in_month,year,change_qty,change_price
127704,22465,Home and Garden,HANGING METAL STAR LANTERN,20101004,24,1.65,1.172676,0,4,277,0,1,40,1,10,4,31,9,1.000000,0.000000
185222,84406B,Home and Garden,CREAM CUPID HEARTS COAT HANGER,20100326,68,3.00,2.206160,4,26,85,0,1,12,4,3,1,31,9,4.666667,-0.076923
18889,20972,Crafts and Hobbies,PINK CREAM FELT CRAFT TRINKET BOX,20091221,12,1.25,1.089766,0,21,355,0,1,52,3,12,4,31,10,-0.666667,0.000000
152943,22781,Home and Garden,GUMBALL MAGAZINE RACK,20101202,2,7.65,6.193592,3,2,336,0,1,48,1,12,4,31,9,0.000000,0.000000
181101,82613A,Home and Garden,METAL SIGNCUPCAKE SINGLE HOOK,20100413,24,1.25,1.034035,1,13,103,0,1,15,2,4,2,30,9,3.800000,0.000000
210980,85168A,Home and Garden,WHITE BAROQUE CARRIAGE CLOCK,20100614,1,9.95,9.384915,0,14,165,0,1,24,2,6,2,30,9,-0.500000,0.000000
200953,84997B,Kitchen and Dining,RED 3 PIECE MINI DOTS CUTLERY SET,20100709,1,3.75,2.270082,4,9,190,0,1,27,2,7,3,31,9,-0.500000,0.000000
102757,22198,Kitchen and Dining,POPCORN HOLDERLARGE,20100812,15,1.65,1.107609,3,12,224,0,1,32,2,8,3,31,9,-0.913793,0.087912
127878,22467,Home and Garden,GUMBALL COAT RACK,20100412,6,2.55,1.531145,0,12,102,0,1,15,2,4,2,30,9,-0.625000,0.000000
212355,85185D,Toys and Games,FROG SOCK PUPPET,20100909,2,2.95,1.745514,3,9,252,0,1,36,2,9,3,30,9,-0.500000,0.000000


store current price and quantity

In [70]:
df.sort_values(by=['Category','Description', 'yyyymmdd'], ascending=[True,True, False], inplace=True)

In [71]:
df['row_number'] = df.groupby('Description').cumcount() + 1

In [72]:
df_lastest = df[df['row_number'] == 1].reset_index(drop=True)

In [73]:
df_lastest.shape

(3888, 21)

In [74]:
df_lastest.head()

,StockCode,Category,Description,yyyymmdd,Quantity,Price,Cost_price,dayofweek,dayofmonth,dayofyear,...,weekday_weekend,weekofyear,weekofmonth,month,quarter,days_in_month,year,change_qty,change_price,row_number
0,22282,Arts and Crafts,12 EGG HOUSE PAINTED WOOD,20101129,2,12.75,6.978710,0,29,333,...,1,48,5,11,4,30,9,-0.875000,0.164384,1
1,21447,Arts and Crafts,12 IVORY ROSE PEG PLACE SETTINGS,20101206,12,1.25,1.214665,0,6,340,...,1,49,1,12,4,31,9,0.714286,0.000000,1
2,21440,Arts and Crafts,12 MINI TOADSTOOL PEGS,20100701,13,1.25,0.974084,3,1,182,...,1,26,1,7,3,31,9,1.600000,0.000000,1
3,20976,Arts and Crafts,12 PENCILS SMALL TUBE POSY,20100813,8,0.65,0.607085,4,13,225,...,1,32,2,8,3,31,9,7.000000,0.000000,1
4,20974,Arts and Crafts,12 PENCILS SMALL TUBE SKULL,20101209,48,0.65,0.489724,3,9,343,...,1,49,2,12,4,31,9,1.086957,0.000000,1


In [75]:
df.columns

Index(['StockCode', 'Category', 'Description', 'yyyymmdd', 'Quantity', 'Price',
       'Cost_price', 'dayofweek', 'dayofmonth', 'dayofyear', 'is_weekend',
       'weekday_weekend', 'weekofyear', 'weekofmonth', 'month', 'quarter',
       'days_in_month', 'year', 'change_qty', 'change_price', 'row_number'],
      dtype='object')

In [76]:
duckdb.query('select Category, count(*), avg(change_qty), avg(change_price) from df group by Category')

┌────────────────────────────┬──────────────┬────────────────────┬───────────────────────┐
│          Category          │ count_star() │  avg(change_qty)   │   avg(change_price)   │
│          varchar           │    int64     │       double       │        double         │
├────────────────────────────┼──────────────┼────────────────────┼───────────────────────┤
│ Office Supplies            │          604 │  3.223020509982275 │   0.05111769456128402 │
│ Tools and Home Improvement │         1864 │ 3.3890398461506255 │   0.00791954689083731 │
│ Food and Beverages         │         4101 │ 2.1048546897923157 │   0.01659604353910767 │
│ Kitchen and Dining         │        42844 │ 3.2876095994817316 │  0.024814354051854327 │
│ Arts and Crafts            │        33997 │   2.87552855408714 │  0.015496941170052974 │
│ Books and Stationery       │        10418 │   5.88584350812584 │     2.391571130547782 │
│ Electronics                │          935 │ 1.9894633199769223 │  0.015036583326544093 │

# 6. Model

split df by category

In [77]:
sorted(df['Category'].unique())

['Arts and Crafts',
 'Automotive',
 'Books and Stationery',
 'Clothing and Accessories',
 'Crafts and Hobbies',
 'Electronics',
 'Food and Beverages',
 'Health and Beauty',
 'Home and Garden',
 'Kitchen and Dining',
 'Office Supplies',
 'Pet',
 'Sports and Outdoors',
 'Tools and Home Improvement',
 'Toys and Games']

In [78]:
df_art_crafts = df[df['Category'] == 'Arts and Crafts']
df_automotive = df[df['Category'] == 'Automotive']
df_book_stationary = df[df['Category'] == 'Books and Stationery']
df_clothing_acc= df[df['Category'] == 'Clothing and Accessories']
df_crafts_hobbies = df[df['Category'] == 'Crafts and Hobbies']
df_electronics = df[df['Category'] == 'Electronics']
df_food_beverages = df[df['Category'] == 'Food and Beverages']
df_health_beauty = df[df['Category'] == 'Health and Beauty']
df_home_garden = df[df['Category'] == 'Home and Garden']
df_kitchen_dining = df[df['Category'] == 'Kitchen and Dining']
df_office_supplies = df[df['Category'] == 'Office Supplies']
df_pet = df[df['Category'] == 'Pet']
df_sports_outdoors = df[df['Category'] == 'Sports and Outdoors']
df_tools_home_improvement = df[df['Category'] == 'Tools and Home Improvement']
df_toys_games = df[df['Category'] == 'Toys and Games']

In [79]:
************* 

SyntaxError: invalid syntax (1047438561.py, line 1)

In [ ]:
X_art_leisure = df_art_leisure.drop(columns=['StockCode','Category','Description','yyyymmdd','Quantity','Price','Cost_price','row_number','change_qty'])
y_art_leisure = df_art_leisure['change_qty']

In [ ]:
print(X_art_leisure.shape, y_art_leisure.shape)

In [ ]:
X_education_office = df_education_office.drop(columns=['StockCode','Category','Description','yyyymmdd','Quantity','Price','Cost_price','row_number','change_qty'])
y_education_office = df_education_office['change_qty']

In [ ]:
print(X_education_office.shape, y_education_office.shape)

In [ ]:
X_fashion_travel = df_fashion_travel.drop(columns=['StockCode','Category','Description','yyyymmdd','Quantity','Price','Cost_price','row_number','change_qty'])
y_fashion_travel = df_fashion_travel['change_qty']

In [ ]:
print(X_fashion_travel.shape, y_fashion_travel.shape)

In [ ]:
X_health_wellness = df_health_wellness.drop(columns=['StockCode','Category','Description','yyyymmdd','Quantity','Price','Cost_price','row_number','change_qty'])
y_health_wellness = df_health_wellness['change_qty']

In [ ]:
print(X_health_wellness.shape, y_health_wellness.shape)

In [ ]:
X_home_lifestyle = df_home_lifestyle.drop(columns=['StockCode','Category','Description','yyyymmdd','Quantity','Price','Cost_price','row_number','change_qty'])
y_home_lifestyle = df_home_lifestyle['change_qty']

In [ ]:
print(X_home_lifestyle.shape, y_home_lifestyle.shape)

In [ ]:
X_tech = df_tech.drop(columns=['StockCode','Category','Description','yyyymmdd','Quantity','Price','Cost_price','row_number','change_qty'])
y_tech = df_tech['change_qty']

In [ ]:
print(X_tech.shape, y_tech.shape)

In [ ]:
***

make scorer

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
rmse_scorer = make_scorer(rmse, greater_is_better=False)

model training (art and leisure)

In [ ]:
rt = DecisionTreeRegressor(random_state=42)

In [ ]:
param_grid = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [3, 5, 10]
}

In [ ]:
grid_search = GridSearchCV(estimator=rt, param_grid=param_grid, cv=3, scoring=rmse_scorer, n_jobs=-1)

In [ ]:
grid_search.fit(X_art_leisure, y_art_leisure)

In [ ]:
grid_search.best_score_

In [ ]:
best_model_art_leisure = grid_search.best_estimator_

model training (education and office)

In [ ]:
rt = DecisionTreeRegressor(random_state=42)

In [ ]:
param_grid = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [3, 5, 10]
}

In [ ]:
grid_search = GridSearchCV(estimator=rt, param_grid=param_grid, cv=3, scoring=rmse_scorer, n_jobs=-1)

In [ ]:
grid_search.fit(X_education_office, y_education_office)

In [ ]:
grid_search.best_score_

In [ ]:
best_model_education_office = grid_search.best_estimator_

model training (fashion and travel)

In [ ]:
rt = DecisionTreeRegressor(random_state=42)

In [ ]:
param_grid = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [3, 5, 10]
}

In [ ]:
grid_search = GridSearchCV(estimator=rt, param_grid=param_grid, cv=3, scoring=rmse_scorer, n_jobs=-1)

In [ ]:
grid_search.fit(X_fashion_travel, y_fashion_travel)

In [ ]:
grid_search.best_score_

In [ ]:
best_model_fashion_travel = grid_search.best_estimator_

model training (health and wellness)

In [ ]:
rt = DecisionTreeRegressor(random_state=42)

In [ ]:
param_grid = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [3, 5, 10]
}

In [ ]:
grid_search = GridSearchCV(estimator=rt, param_grid=param_grid, cv=3, scoring=rmse_scorer, n_jobs=-1)

In [ ]:
grid_search.fit(X_health_wellness, y_health_wellness)

In [ ]:
grid_search.best_score_

In [ ]:
best_model_health_wellness = grid_search.best_estimator_

model training (home and lifestyle)

In [ ]:
rt = DecisionTreeRegressor(random_state=42)

In [ ]:
param_grid = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [3, 5, 10]
}

In [ ]:
grid_search = GridSearchCV(estimator=rt, param_grid=param_grid, cv=3, scoring=rmse_scorer, n_jobs=-1)

In [ ]:
grid_search.fit(X_home_lifestyle, y_home_lifestyle)

In [ ]:
grid_search.best_score_

In [ ]:
best_model_home_lifestyle = grid_search.best_estimator_

model training (tech)

In [ ]:
rt = DecisionTreeRegressor(random_state=42)

In [ ]:
param_grid = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [3, 5, 10]
}

In [ ]:
grid_search = GridSearchCV(estimator=rt, param_grid=param_grid, cv=3, scoring=rmse_scorer, n_jobs=-1)

In [ ]:
grid_search.fit(X_tech, y_tech)

In [ ]:
grid_search.best_score_

In [ ]:
best_model_tech = grid_search.best_estimator_

dict all model

In [ ]:
model_dict = {
    'Arts and Leisure': best_model_art_leisure,
    'Education and Office': best_model_education_office,
    'Fashion and Travel': best_model_fashion_travel,
    'Health and Wellness': best_model_health_wellness,
    'Home and Lifestyle': best_model_home_lifestyle,
    'Technology and More': best_model_tech,
}

# 7. Optimize

In [ ]:
df_final = pd.DataFrame()

In [ ]:
for index, row in df_lastest.iterrows():
    
    df_optimize = df_lastest[index:index+1][['dayofweek','dayofmonth','dayofyear','is_weekend',
                                        'weekday_weekend','weekofyear','weekofmonth','month',
                                        'quarter','days_in_month','year']]
    multipliers = [round(x * 0.01, 2) for x in range(-50, 51)]
    df_optimize = df_optimize.loc[np.repeat(df_optimize.index.values, len(multipliers))]
    df_optimize['change_price'] = multipliers

    change_price_list = []
    for i in range(df_optimize.shape[0]):
        if row['Category'] == 'Arts and Leisure':
            demand_change = best_model_art_leisure.predict(df_optimize.iloc[[i]])
        elif row['Category'] == 'Education and Office':
            demand_change = best_model_education_office.predict(df_optimize.iloc[[i]])
        elif row['Category'] == 'Fashion and Travel':
            demand_change = best_model_fashion_travel.predict(df_optimize.iloc[[i]])
        elif row['Category'] == 'Health and Wellness':
            demand_change = best_model_health_wellness.predict(df_optimize.iloc[[i]])
        elif row['Category'] == 'Home and Lifestyle':
            demand_change = best_model_home_lifestyle.predict(df_optimize.iloc[[i]])
        elif row['Category'] == 'Technology and More':
            demand_change = best_model_tech.predict(df_optimize.iloc[[i]])        

        change_price_list.append(demand_change[0])
        
    df_optimize['change_qty'] = change_price_list

    initial_price = df_lastest[df_lastest['StockCode'] == row['StockCode']]['Price'].iloc[0]
    initial_cost_price = df_lastest[df_lastest['StockCode'] == row['StockCode']]['Cost_price'].iloc[0]
    initial_quantity = df_lastest[df_lastest['StockCode'] == row['StockCode']]['Quantity'].iloc[0]

    df_optimize['price_new'] = initial_price*(1+df_optimize['change_price'])
    df_optimize['qty_new'] = initial_quantity*(1+df_optimize['change_qty'])
    df_optimize['additional_profit'] = df_optimize['qty_new']*(df_optimize['price_new']-initial_cost_price)
    df_optimize = df_optimize[df_optimize['additional_profit'] == df_optimize['additional_profit'].max()]
    
    df_final_append = df_lastest[index:index+1][['StockCode','Category','Description','yyyymmdd','Quantity','Price']]
    df_final_append['change_price'] = df_optimize['change_price'].values[0]
    df_final_append['change_qty'] = df_optimize['change_qty'].values[0]
    df_final_append['price_new'] = df_optimize['price_new'].values[0]
    df_final_append['qty_new'] = df_optimize['qty_new'].values[0]
    df_final_append['additional_profit'] = df_optimize['additional_profit'].values[0]

    df_final = pd.concat([df_final, df_final_append], ignore_index=True)

In [ ]:
df_final